In [18]:
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_community.agent_toolkits import create_sql_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from pyprojroot import here
from operator import itemgetter
import chromadb
import os
from dotenv import load_dotenv
load_dotenv()

True

### Comparing RAG and SQL Agent for different questions

In [ ]:
question1 = "What is the lowest rated movies with more than 1 million votes?"
question2 = "How many movies have Tom Hanks played in?"
question3 = "Name two actors that have played together in more than 4 movies."
question4 = "Name two popular movies about chess."
question5 = "

### SQL

In [ ]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


In [ ]:
sqldb_directory = here("data/db/sql/imdb.db")
db = SQLDatabase.from_uri(f"sqlite:///{sqldb_directory}")

### Simple SQL query writer

In [22]:
system_role = """Given the following user question, corresponding SQL query, and SQL result, answer the user question.\n
    Question: {question}\n
    SQL Query: {query}\n
    SQL Result: {result}\n
    Answer:
    """

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
answer_prompt = PromptTemplate.from_template(system_role)
answer = answer_prompt | llm | StrOutputParser()

chain = (
    RunnablePassthrough.assign(query=write_query)
    .assign(result=itemgetter("query") | execute_query)
    .assign(answer=answer)
)



In [ ]:
system_role = """Given the following user question, corresponding SQL query, and SQL result, answer the user question.\n
    Question: {question}\n
    SQL Query: {query}\n
    SQL Result: {result}\n
    Answer:
    """

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(
    llm, db)
answer_prompt = PromptTemplate.from_template(
    system_role)
answer = answer_prompt | llm | StrOutputParser()

chain = (
    RunnablePassthrough.assign(query=write_query)
    .assign(result=itemgetter("query") | execute_query)
    .assign(answer=answer)
)

In [24]:
response1 = chain.invoke({"question": question1})


In [25]:
response1

{'question': 'What is the lowest rated movies with more than 1 million votes?',
 'query': 'SELECT tb."primaryTitle", tr."averageRating", tr."numVotes"\nFROM "title.basics" tb\nJOIN "title.ratings" tr ON tb.tconst = tr.tconst\nWHERE tr."numVotes" > 1000000\nORDER BY tr."averageRating" ASC\nLIMIT 5;',
 'result': "[('The Hunger Games', 7.2, 1031437), ('Titanic', 7.9, 1325403), ('Iron Man', 7.9, 1159536), ('Avatar', 7.9, 1419190), ('The Avengers', 8.0, 1494463)]",
 'answer': 'The lowest rated movies with more than 1 million votes are:\n1. The Hunger Games with an average rating of 7.2 and 1,031,437 votes\n2. Titanic with an average rating of 7.9 and 1,325,403 votes\n3. Iron Man with an average rating of 7.9 and 1,159,536 votes'}

In [26]:
response = chain.invoke({"question": question2})
response

{'question': 'How many movies have Tom Hanks played in?',
 'query': 'SELECT COUNT(*) \nFROM "title.principals" \nWHERE nconst = \'nm0000158\' AND category = \'actor\';',
 'result': '[(210,)]',
 'answer': 'Tom Hanks has played in a total of 210 movies.'}

In [27]:
response = chain.invoke({"question": question3})
response

{'question': 'Name two actors that have played together in more than 4 movies.',
 'query': 'SELECT nb1."primaryName", nb2."primaryName", COUNT(*) AS movies_played_together\nFROM "title.principals" tp1\nJOIN "title.principals" tp2 ON tp1.tconst = tp2.tconst AND tp1.nconst < tp2.nconst\nJOIN "name.basics" nb1 ON tp1.nconst = nb1.nconst\nJOIN "name.basics" nb2 ON tp2.nconst = nb2.nconst\nWHERE nb1."primaryProfession" LIKE \'%actor%\' AND nb2."primaryProfession" LIKE \'%actor%\'\nGROUP BY nb1."primaryName", nb2."primaryName"\nHAVING movies_played_together > 4\nORDER BY movies_played_together DESC\nLIMIT 2;\n```',
 'result': 'Error: (sqlite3.ProgrammingError) You can only execute one statement at a time.\n[SQL: SELECT nb1."primaryName", nb2."primaryName", COUNT(*) AS movies_played_together\nFROM "title.principals" tp1\nJOIN "title.principals" tp2 ON tp1.tconst = tp2.tconst AND tp1.nconst < tp2.nconst\nJOIN "name.basics" nb1 ON tp1.nconst = nb1.nconst\nJOIN "name.basics" nb2 ON tp2.nconst = 

In [28]:
response = chain.invoke({"question": question4}, config={"verbose": True})
response

{'question': 'Name two popular movies about chess.',
 'query': 'SELECT tb."primaryTitle", tb.genres\nFROM "title.basics" tb\nWHERE tb.genres LIKE \'%Chess%\'\nLIMIT 2;',
 'result': '',
 'answer': 'Two popular movies about chess are "Searching for Bobby Fischer" and "Queen of Katwe".'}

### LangChain SQL Agent

In [29]:
agent_executor = create_sql_agent(
    llm, 
    db=db, 
    agent_type="openai-tools", 
    verbose=True, 
    system_message="You are an advanced SQL agent. Your task is to generate accurate and efficient SQL queries to answer user questions based on the database schema and data. The database is the IMDB database with multiple tables and information about movies and tv shows. Do not use the id of actors and movies in the result (e.g. tt00001 and nm000001), instead find the real names."
)

In [30]:
response1_sql = agent_executor.invoke({"input": question1})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


name, name.basics, title, title.akas, title.basics, title.crew, title.episode, title.principals, title.ratings
Invoking: `sql_db_schema` with `{'table_names': 'title.ratings'}`



CREATE TABLE "title.ratings" (
	tconst TEXT, 
	"averageRating" FLOAT, 
	"numVotes" BIGINT
)

/*
3 rows from title.ratings table:
tconst	averageRating	numVotes
tt0000001	5.7	2127
tt0000002	5.6	286
tt0000003	6.5	2163
*/
Invoking: `sql_db_query` with `{'query': 'SELECT tconst, averageRating, numVotes FROM "title.ratings" WHERE numVotes > 100000 ORDER BY averageRating LIMIT 10'}`


[('tt10888594', 1.8, 180420), ('tt0799949', 2.4, 110955), ('tt12915716', 2.7, 135118), ('tt1073498', 2.8, 112932), ('tt10886166', 3.3, 102606), ('tt0327554', 3.4, 128307), ('tt0118688', 3.8, 273952), ('tt0938283', 4.0, 177584), ('tt6027920', 4.0, 270011), ('tt12262202', 4.2, 129817)]The lowest rated movies with more than 1 million votes are:

1. Mo

In [31]:
response2_sql = agent_executor.invoke({"input": question2})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


name, name.basics, title, title.akas, title.basics, title.crew, title.episode, title.principals, title.ratings
Invoking: `sql_db_schema` with `{'table_names': 'name, title.principals'}`



CREATE TABLE "title.principals" (
	tconst TEXT, 
	ordering BIGINT, 
	nconst TEXT, 
	category TEXT, 
	job TEXT, 
	characters TEXT
)

/*
3 rows from title.principals table:
tconst	ordering	nconst	category	job	characters
tt0000001	1	nm1588970	self	None	["Self"]
tt0000001	2	nm0005690	director	None	None
tt0000001	3	nm0005690	producer	producer	None
*/


CREATE TABLE name (
	nconst TEXT, 
	"primaryName" TEXT, 
	"birthYear" TEXT, 
	"deathYear" TEXT, 
	"primaryProfession" TEXT, 
	"knownForTitles" TEXT
)

/*
3 rows from name table:
nconst	primaryName	birthYear	deathYear	primaryProfession	knownForTitles
nm0000001	Fred Astaire	1899	1987	actor,miscellaneous,producer	tt0072308,tt0050419,tt0027125,tt0031983
nm0000002	Lauren Bac

In [32]:
response3_sql = agent_executor.invoke({"input": question3})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


name, name.basics, title, title.akas, title.basics, title.crew, title.episode, title.principals, title.ratings
Invoking: `sql_db_schema` with `{'table_names': 'name, title, title.principals'}`



CREATE TABLE "title.principals" (
	tconst TEXT, 
	ordering BIGINT, 
	nconst TEXT, 
	category TEXT, 
	job TEXT, 
	characters TEXT
)

/*
3 rows from title.principals table:
tconst	ordering	nconst	category	job	characters
tt0000001	1	nm1588970	self	None	["Self"]
tt0000001	2	nm0005690	director	None	None
tt0000001	3	nm0005690	producer	producer	None
*/


CREATE TABLE name (
	nconst TEXT, 
	"primaryName" TEXT, 
	"birthYear" TEXT, 
	"deathYear" TEXT, 
	"primaryProfession" TEXT, 
	"knownForTitles" TEXT
)

/*
3 rows from name table:
nconst	primaryName	birthYear	deathYear	primaryProfession	knownForTitles
nm0000001	Fred Astaire	1899	1987	actor,miscellaneous,producer	tt0072308,tt0050419,tt0027125,tt0031983
nm0000002	Lau

In [33]:
response4_sql = agent_executor.invoke({"input": question4})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


name, name.basics, title, title.akas, title.basics, title.crew, title.episode, title.principals, title.ratings
Invoking: `sql_db_schema` with `{'table_names': 'title.akas, title.basics'}`



CREATE TABLE "title.akas" (
	"titleId" TEXT, 
	ordering BIGINT, 
	title TEXT, 
	region TEXT, 
	language TEXT, 
	types TEXT, 
	attributes TEXT, 
	"isOriginalTitle" BIGINT
)

/*
3 rows from title.akas table:
titleId	ordering	title	region	language	types	attributes	isOriginalTitle
tt0000001	1	Carmencita	None	None	original	None	1
tt0000001	2	Carmencita	DE	None	None	literal title	0
tt0000001	3	Carmencita	US	None	imdbDisplay	None	0
*/


CREATE TABLE "title.basics" (
	tconst TEXT, 
	"titleType" TEXT, 
	"primaryTitle" TEXT, 
	"originalTitle" TEXT, 
	"isAdult" TEXT, 
	"startYear" TEXT, 
	"endYear" TEXT, 
	"runtimeMinutes" TEXT, 
	genres TEXT
)

/*
3 rows from title.basics table:
tconst	titleType	primaryTitle	originalTitl

### RAG

In [34]:
chroma_client = chromadb.PersistentClient(path=here("data/db/chroma"))
collection_name = "imdb"
collection = chroma_client.get_collection(collection_name)

TypeError: unsupported operand type(s) for +: 'PosixPath' and 'str'

In [ ]:
embedding_client = OpenAIEmbeddings(
    model="text-embedding-3-small",
    # With the `text-embedding-3` class
    # of models, you can specify the size
    # of the embeddings you want returned.
    # dimensions=1024
)

In [ ]:
question1_embedding = embedding_client.embed_query(question1)
question1_vector_search = collection.query(
    query_embeddings = question1_embedding,
    n_results=3 #top_k
)
vector_db_results = str(question1_vector_search["documents"])

In [ ]:
vector_db_results

'[[\'\\nTitle: 537 Votes (2020)\\nType: movie\\nRuntime: 109.0 minutes\\nGenres: Documentary\\nIMDb Rating: 7.2 (Votes: 930)\\n\\nCast & Crew:\\nName:Ben Affleck 1972, characters:["Self"], job:archive_footage\\nName:Fernand Amandi None, characters:["Self"], job:self\\nName:Ann Louise Bardach None, characters:["Self"], job:self\\nName:Mitchell Berger None, characters:["Self"], job:self\\nName:Brad Blakeman None, characters:["Self"], job:self\\nName:Jon Bon Jovi 1962, characters:["Self"], job:archive_footage\\nName:George W. Bush 1946, characters:["Self"], job:archive_footage\\nName:Fidel Castro 1926, characters:["Self"], job:archive_footage\\nName:Dave Chappelle 1973, characters:["Self"], job:archive_footage\\nName:Bill Clinton 1946, characters:["Self"], job:archive_footage\\nName:Billy Corben 1978, characters:None, job:director\\nName:Billy Corben 1978, characters:None, job:producer\\nName:Alfred Spellman 1978, characters:None, job:producer\\nName:Brian Robertson None, characters:None,

In [ ]:


system_role = """Given the following user question, and corresponding results from the vector databse, answer the user question.\n
If the vector search results are not related to the question, say "I don't know".\n

Question: {question}\n
Vector search results: {result}\n
Answer:
"""
    
answer_prompt = PromptTemplate.from_template(system_role)
answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(embedding=lambda x: embedding_client.embed_query(x["question"]))
    .assign(vector_search=lambda x: collection.query(query_embeddings=x["embedding"], n_results=3))
    .assign(result=lambda x: str(x["vector_search"]["documents"]))
    .assign(answer=answer)
)

In [ ]:
response1 = chain.invoke({"question": question1})
print(response1["result"])
print(response1["answer"])

[['\nTitle: 537 Votes (2020)\nType: movie\nRuntime: 109.0 minutes\nGenres: Documentary\nIMDb Rating: 7.2 (Votes: 930)\n\nCast & Crew:\nName:Ben Affleck 1972, characters:["Self"], job:archive_footage\nName:Fernand Amandi None, characters:["Self"], job:self\nName:Ann Louise Bardach None, characters:["Self"], job:self\nName:Mitchell Berger None, characters:["Self"], job:self\nName:Brad Blakeman None, characters:["Self"], job:self\nName:Jon Bon Jovi 1962, characters:["Self"], job:archive_footage\nName:George W. Bush 1946, characters:["Self"], job:archive_footage\nName:Fidel Castro 1926, characters:["Self"], job:archive_footage\nName:Dave Chappelle 1973, characters:["Self"], job:archive_footage\nName:Bill Clinton 1946, characters:["Self"], job:archive_footage\nName:Billy Corben 1978, characters:None, job:director\nName:Billy Corben 1978, characters:None, job:producer\nName:Alfred Spellman 1978, characters:None, job:producer\nName:Brian Robertson None, characters:None, job:composer\nName:Jon

In [ ]:
reseponse2 = chain.invoke({"question": question2})
print(reseponse2["result"])
print(reseponse2["answer"])

[['\nTitle: Cast Away (2000)\nType: movie\nRuntime: 143.0 minutes\nGenres: Adventure,Drama,Romance\nIMDb Rating: 7.8 (Votes: 658740)\n\nCast & Crew:\nName:Tom Hanks 1956, characters:["Chuck Noland"], job:actor\nName:Helen Hunt 1963, characters:["Kelly Frears"], job:actress\nName:Paul Sanchez None, characters:["Ramon"], job:actor\nName:Lari White 1965, characters:["Bettina Peterson"], job:actress\nName:Leonid Citer 1960, characters:["Fyodor"], job:actor\nName:David Allen Brooks 1947, characters:["Dick Peterson"], job:actor\nName:Yelena Popovic None, characters:["Beautiful Russian Woman"], job:actress\nName:Valentina Ananina 1933, characters:["Russian Babushka"], job:actress\nName:Semion Sudarikov None, characters:["Nicolai"], job:actor\nName:Peter Von Berg 1947, characters:["Yuri"], job:actor\nName:Robert Zemeckis 1952, characters:None, job:director\nName:William Broyles Jr. 1944, characters:None, job:writer\nName:Tom Hanks 1956, characters:None, job:producer\nName:Jack Rapke None, char

In [ ]:
reseponse3 = chain.invoke({"question": question3})
print(reseponse3["result"])
print(reseponse3["answer"])

[['\nTitle: Four Friends (1981)\nType: movie\nRuntime: 114.0 minutes\nGenres: Comedy,Drama\nIMDb Rating: 7.0 (Votes: 2082)\n\nCast & Crew:\nName:Craig Wasson 1954, characters:["Danilo"], job:actor\nName:Jodi Thelen 1962, characters:["Georgia"], job:actress\nName:Michael Huddleston 1952, characters:["David"], job:actor\nName:Jim Metzler 1951, characters:["Tom"], job:actor\nName:Scott Hardt None, characters:["Young Danilo"], job:actor\nName:Elizabeth Lawrence 1922, characters:["Mrs. Prozor"], job:actress\nName:Miklos Simon None, characters:["Mr. Prozor"], job:actor\nName:Michael Kovacs None, characters:["Prozor Neighbor"], job:actor\nName:Beatrice Fredman 1918, characters:["Mrs. Zoldos"], job:actress\nName:Pier Calabria None, characters:["Conductor"], job:actress\nName:Arthur Penn 1922, characters:None, job:director\nName:Steve Tesich 1942, characters:None, job:writer\nName:Gene Lasko 1929, characters:None, job:producer\nName:Arthur Penn 1922, characters:None, job:producer\nName:Elizabet

In [ ]:
reseponse4 = chain.invoke({"question": question4})
print(reseponse4["result"])
print(reseponse4["answer"])

[['\nTitle: Computer Chess (2013)\nType: movie\nRuntime: 92.0 minutes\nGenres: Comedy\nIMDb Rating: 6.2 (Votes: 4906)\n\nCast & Crew:\nName:Kriss Schludermann None, characters:["ADVANTAGE Member"], job:actor\nName:Tom Fletcher None, characters:["DEEP SPEED Member"], job:actor\nName:Wiley Wiggins 1976, characters:["Beuscher"], job:actor\nName:Patrick Riester None, characters:["Bishton"], job:actor\nName:Kevin Bewersdorf None, characters:["Cameraman"], job:actor\nName:Gene Williams None, characters:["MONSIEUR D\'ECHECS member"], job:actor\nName:Jim Lewis None, characters:["John"], job:actor\nName:Cole Noppenberg None, characters:["CAPA X Member"], job:actor\nName:Myles Paige None, characters:["Papageorge"], job:actor\nName:Gerald Peary None, characters:["Henderson"], job:actor\nName:Andrew Bujalski 1977, characters:None, job:director\nName:Andrew Bujalski 1977, characters:None, job:writer\nName:Houston King None, characters:None, job:producer\nName:Alex Lipschultz None, characters:None, 